# **Capstone 2 : Final Script**



* Shrinidhi Sudhir   
* Duyen Do
* Lakshmi Kambathanahally Lakshminarasapa
* Jessica Nguyen



### **Detailed Code Analysis for CLABSI Data Exploration and Preprocessing**


This report breaks down the Python code used for data exploration, preprocessing, and preparation for machine learning modeling. The dataset contains clinical data relevant to CLABSI (Central Line Associated Blood Stream Infections).

### Setup

First, we import necessary libraries and load the dataset.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_pipeline_imblearn
from sklearn.metrics import precision_recall_curve


# Load the dataset
data = pd.read_csv('/content/bzan6361_clabsi_1.csv')


<ipython-input-1-4fa8efa7ce64>:21: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/bzan6361_clabsi_1.csv')


**Data Inspection**

Here, we display the first few entries of the dataset and basic information such as column datatypes and memory usage.

In [2]:
# Display the first few rows of the DataFrame
data.head()

# Display basic information about the DataFrame
print("Basic Information about the DataFrame:")
print(data.info())



Basic Information about the DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14236 entries, 0 to 14235
Columns: 278 entries, PatientKey to ICUDaysLast2
dtypes: bool(19), float64(19), int64(208), object(32)
memory usage: 28.4+ MB
None


**Understanding the Target Variable**

We explore the target variable HasCLABSI which indicates the presence of an infection.

In [3]:
# Understanding the target variable
unique_values_count = data['HasCLABSI'].nunique()
print("\nNumber of Unique Values in 'HasClabsi' Column:", unique_values_count)

unique_values_counts = data['HasCLABSI'].value_counts()
print("\nCount of Each Unique Value in 'HasClabsi' Column:")
print(unique_values_counts)


Number of Unique Values in 'HasClabsi' Column: 2

Count of Each Unique Value in 'HasClabsi' Column:
HasCLABSI
False    14184
True        52
Name: count, dtype: int64


### **Zoom In Analysis**

**Data Cleaning**



*   **Remove Highly Correlated Columns** - We define a function to remove highly correlated numeric columns to reduce multicollinearity.
*   **Extract Columns with Max Number Suffix** - This function extracts columns that have numeric suffixes, selecting the maximum number for each group.

In [4]:
# Define function to remove highly correlated columns, excluding boolean columns
def remove_highly_correlated_columns(df, threshold=0.8):
    corr_matrix = df.corr().abs()
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]
    to_drop = [column for column in to_drop if column not in df.select_dtypes(include=['bool']).columns]
    return df.drop(to_drop, axis=1), to_drop



In [5]:
# Define function to extract columns with the max number suffix
def extract_max_number_suffix(df, pattern=r'(\D+)(\d*)$'):
    column_groups = {}
    for column in df.columns:
        match = re.match(pattern, column)
        if match:
            prefix, number = match.groups()
            if prefix in column_groups:
                if number.isdigit():
                    column_groups[prefix] = max(column_groups[prefix], int(number))
            else:
                column_groups[prefix] = int(number) if number.isdigit() else 0
    columns_to_keep = [prefix + str(number) if number != 0 else prefix for prefix, number in column_groups.items()]
    return df[columns_to_keep]

**Data Transformation**



*  Dimensionality Reduction - We apply the previously defined functions to reduce the dimensionality of the DataFrame.
*  Output Changes in Dimensionality - Finally, we print the changes in dimensionality and the columns that were dropped.




In [6]:
# Apply functions to reduce the dimensionality of the DataFrame
non_boolean_numerical_columns = data.select_dtypes(include=[np.number]).columns.difference(data.select_dtypes(include=['bool']).columns)
data_numeric, dropped_columns = remove_highly_correlated_columns(data[non_boolean_numerical_columns])

# Directly concatenate the processed numeric columns with the original boolean columns from `data`
boolean_columns = data.select_dtypes(include=['bool']).columns.tolist()
data_final = pd.concat([data_numeric, data[boolean_columns]], axis=1)

# Output the changes in dimensionality
num_columns_before = len(data.columns)
num_columns_after = len(data_final.columns)
print("\nColumns before reduction:", num_columns_before)
print("Columns after reduction:", num_columns_after)
print("Dropped columns:", dropped_columns)



Columns before reduction: 278
Columns after reduction: 168
Dropped columns: ['AlteplaseAdministeredLast3', 'AlteplaseAdministeredLast30', 'AlteplaseAdministeredLast5', 'CHGBathsLast3', 'CHGBathsLast30', 'CHGBathsLast5', 'CapChangesLast7', 'CountMedicationsLast2', 'CountMedicationsLast3', 'CountMedicationsLast30', 'CountMedicationsLast5', 'DiagnosisClotLast3', 'DressingChangesLast7', 'FlushedLast4', 'FlushedLast7', 'HCLAdministeredLast2', 'HCLAdministeredLast3', 'HCLAdministeredLast30', 'HCLAdministeredLast5', 'ICUDaysLast2', 'ICUDaysLast3', 'ICUDaysLast30', 'ICUDaysLast5', 'LineDaysMultiLumen', 'LineDaysPort', 'LineDaysUpperArm', 'MedicationsInjectedLast3', 'MedicationsInjectedLast30', 'MedicationsInjectedLast5', 'MedsAcidSuppTherapyLast2', 'MedsAcidSuppTherapyLast3', 'MedsAcidSuppTherapyLast30', 'MedsAcidSuppTherapyLast5', 'MedsBowelRegimenLast3', 'MedsBowelRegimenLast30', 'MedsBowelRegimenLast5', 'MedsCentralTPNLast3', 'MedsCentralTPNLast30', 'MedsCentralTPNLast5', 'MedsFatEmulsionL

**Categorization of Data by Type**


Before addressing missing values, it is important to categorize columns by their data type for appropriate handling.

In [7]:
int_columns = data_final.select_dtypes(include='int64')
object_columns = data_final.select_dtypes(include='object')
bool_columns = data_final.select_dtypes(include='bool')
float_columns = data_final.select_dtypes(include='float64')
datetime_columns = data_final.select_dtypes(include='datetime64[ns]')


**Identifying Missing Values**

We systematically identify missing values across different data types to ensure completeness of the dataset.

In [8]:
# Displaying columns with missing values and their count for each data type
print("Integer Columns with Missing Values:")
for column in int_columns.columns[int_columns.isnull().any()]:
    print(f"{column}: {int_columns[column].isnull().sum()} missing values")

print("\nObject Columns with Missing Values:")
for column in object_columns.columns[object_columns.isnull().any()]:
    print(f"{column}: {object_columns[column].isnull().sum()} missing values")

print("\nBoolean Columns with Missing Values:")
for column in bool_columns.columns[bool_columns.isnull().any()]:
    print(f"{column}: {bool_columns[column].isnull().sum()} missing values")

print("\nFloat Columns with Missing Values:")
for column in float_columns.columns[float_columns.isnull().any()]:
    print(f"{column}: {float_columns[column].isnull().sum()} missing values")

print("\nDatetime Columns with Missing Values:")
for column in datetime_columns.columns[datetime_columns.isnull().any()]:
    print(f"{column}: {datetime_columns[column].isnull().sum()} missing values")


Integer Columns with Missing Values:

Object Columns with Missing Values:

Boolean Columns with Missing Values:

Float Columns with Missing Values:
CapChangesLast10: 1472 missing values
CapChangesLast4: 1472 missing values
CapChangesLastToday: 1472 missing values
DaysToCLABSI: 13143 missing values
DressingChangesLast10: 1472 missing values
DressingChangesLast4: 1472 missing values
DressingChangesLastToday: 1472 missing values
FlushedLast10: 1472 missing values
FlushedToday: 1472 missing values
LastAnc: 198 missing values
LastAncDelta: 1134 missing values
TubingChangesLast10: 1472 missing values
TubingChangesLast4: 1472 missing values
TubingChangesLastToday: 1472 missing values

Datetime Columns with Missing Values:


**Handling Missing Values**

Considering the skewness of the data, missing values in numerical columns are filled with the median of the respective group based on the HasCLABSI status. This approach mitigates the impact of outliers affecting the imputation.

In [9]:
# Fill missing values in object columns with 'NA'
data_final[object_columns.columns] = data_final[object_columns.columns].fillna('NA')

# Filling missing values in float and integer columns with medians based on 'HasCLABSI' status
for column in float_columns.columns:
    data_final.loc[data_final['HasCLABSI'] == True, column] = data_final.loc[data_final['HasCLABSI'] == True, column].fillna(data_final.loc[data_final['HasCLABSI'] == True, column].median())
    data_final.loc[data_final['HasCLABSI'] == False, column] = data_final.loc[data_final['HasCLABSI'] == False, column].fillna(data_final.loc[data_final['HasCLABSI'] == False, column].median())

for column in int_columns.columns:
    data_final.loc[data_final['HasCLABSI'] == True, column] = data_final.loc[data_final['HasCLABSI'] == True, column].fillna(data_final.loc[data_final['HasCLABSI'] == True, column].median())
    data_final.loc[data_final['HasCLABSI'] == False, column] = data_final.loc[data_final['HasCLABSI'] == False, column].fillna(data_final.loc[data_final['HasCLABSI'] == False, column].median())

# Re-checking for any remaining missing values
missing_values_after_filling = data_final.isnull().sum().sort_values(ascending=False)
if missing_values_after_filling.empty:
    print("No missing values remain.")
else:
    print("Missing values after filling:")
    print(missing_values_after_filling)


Missing values after filling:
AlteplaseAdministeredLast15    0
SurgeriesAbdominalLast5        0
PICUDaysLast30                 0
PastCLABSIs                    0
PatientKey                     0
                              ..
FlushedToday                   0
HCLAdministeredLast15          0
HospitalDay                    0
ICUDaysLast15                  0
DiagnosisSwellingLast2         0
Length: 168, dtype: int64


**Date Handling and Sorting**


The Date column is converted to datetime format, and the data is sorted by PatientKey and Date to prepare for shifting operations in creating lagged variables.

In [10]:
# Ensure 'Date' column is in datetime format and sort data
data_final['Date'] = pd.to_datetime(data['Date'])
data_final.sort_values(by=['PatientKey', 'Date'], inplace=True)


**Creating Predictive Variables**


Creating variables to predict CLABSI occurrences within one and three days using data shifting techniques.

In [11]:
# Ensure 'Date' column is in datetime format
data_final['Date'] = pd.to_datetime(data['Date'])

# Sort by 'PatientKey' and 'Date'
data_final.sort_values(by=['PatientKey', 'Date'], inplace=True)

# Assuming CLABSI is recorded the day it occurs and the day prior,
# we shift the 'HasCLABSI' column by 1 to get the status of CLABSI for the next day
data_final['HasCLABSI_NextDay'] = data_final.groupby('PatientKey')['HasCLABSI'].shift(-1)

# Create a new target variable for prediction of CLABSI in the next three days
# For simplicity, we'll say a patient is at risk if they will have CLABSI either tomorrow or the two following days
data_final['HasCLABSI_Next3Days'] = data_final.groupby('PatientKey')['HasCLABSI'].shift(-1) | data_final.groupby('PatientKey')['HasCLABSI'].shift(-2) | data_final.groupby('PatientKey')['HasCLABSI'].shift(-3)

# Handle the NaN values which we'll assume means no CLABSI within the next 3 days
data_final['HasCLABSI_Next3Days'].fillna(False, inplace=True)


### **Modeling Design**

**Preparing Data for Predictive Modeling and Evaluating Models**

This section describes the process of preparing the dataset for machine learning and evaluating various models to predict the occurrence of CLABSI within three days.

**Listing Boolean Columns**


First, identify all boolean columns in the data_final DataFrame. These columns represent categorical data in binary format.

In [12]:
# List the boolean columns in data_final
boolean_columns_in_data_final = data_final.select_dtypes(include=['bool']).columns.tolist()
print(boolean_columns_in_data_final)


['HasCLABSI', 'HasFutureEncounterCLABSI', 'HadPreviousCLABSI', 'HasRecentLDAFlowsheetRecords', 'DiagnosisLeukemiaLast30', 'DiagnosisLeukemiaLast15', 'DiagnosisLeukemiaLast5', 'DiagnosisLeukemiaLast3', 'DiagnosisLeukemiaLast2', 'DiagnosisTransplantLast30', 'DiagnosisTransplantLast15', 'DiagnosisTransplantLast5', 'DiagnosisTransplantLast3', 'DiagnosisTransplantLast2', 'DiagnosisSwellingLast30', 'DiagnosisSwellingLast15', 'DiagnosisSwellingLast5', 'DiagnosisSwellingLast3', 'DiagnosisSwellingLast2', 'HasCLABSI_Next3Days']


**Preparing Features and Labels**


Prepare the feature matrix X and the target vector y by excluding specific columns that either do not contribute to prediction or leak future information.

In [13]:
# Define columns to exclude from features including datetime or identifiers related to the outcome
columns_to_exclude = ['HasCLABSI', 'HasCLABSI_NextDay', 'HasCLABSI_Next3Days', 'Date', 'DaysToCLABSI']
# Filter out the columns from 'columns_to_exclude' if they exist in 'data_final'
filtered_columns = [col for col in columns_to_exclude if col in data_final.columns]

# Prepare the feature matrix X and the target variable y
X = data_final.drop(filtered_columns, axis=1, errors='ignore')
y = data_final['HasCLABSI_Next3Days'].astype(int)  # Convert the target variable to integer for modeling

# Display the final shape of X and y
print("Features shape:", X.shape)
print("Target shape:", y.shape)


Features shape: (14236, 166)
Target shape: (14236,)


**Splitting the Dataset**


Split the data into training and testing sets to ensure the model is evaluated on unseen data, enhancing the generalizability of the model predictions.

In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


**Defining and Evaluating Models**

Define multiple machine learning models to assess which performs best in predicting the risk of CLABSI within the next three days.

In [15]:
# Define a dictionary of models
additional_models = {
    "Neural Network": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

# Iterate over each model, fit, and evaluate
for name, model in additional_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Display results for each model
    print(f"Results for {name}:")
    print(classification_report(y_test, y_pred))
    print(f"AUC: {roc_auc_score(y_test, y_proba)}\n")


Results for Neural Network:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      2827
           1       0.00      0.00      0.00        21

    accuracy                           0.99      2848
   macro avg       0.50      0.50      0.50      2848
weighted avg       0.99      0.99      0.99      2848

AUC: 0.5



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Results for K-Nearest Neighbors:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2827
           1       0.20      0.14      0.17        21

    accuracy                           0.99      2848
   macro avg       0.60      0.57      0.58      2848
weighted avg       0.99      0.99      0.99      2848

AUC: 0.871831151986794

Results for Naive Bayes:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98      2827
           1       0.04      0.19      0.06        21

    accuracy                           0.96      2848
   macro avg       0.52      0.58      0.52      2848
weighted avg       0.99      0.96      0.97      2848

AUC: 0.7649872824970101

Results for Decision Tree:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2827
           1       0.59      0.62      0.60        21

    accuracy                           0.99     

### **Evaluation Plan**

**Evaluation Plan**

This section outlines the evaluation strategies for assessing the performance of predictive models developed to forecast CLABSI occurrences. The plan addresses handling overfitting, dealing with imbalanced data, and selecting appropriate metrics considering the clinical implications of predictions.


**Handling Overfitting**

To prevent overfitting and ensure the model generalizes well to new data, we utilize techniques like cross-validation and hyperparameter tuning. We'll use the Decision Tree model as an example:

In [16]:
# Decision Tree with GridSearchCV for hyperparameter tuning
parameters = {'max_depth': [10, 20, 30], 'min_samples_split': [2, 5, 10]}
dt = DecisionTreeClassifier(random_state=42)
clf = GridSearchCV(dt, parameters, cv=5)
clf.fit(X_train, y_train)

# Best model parameters
print("Best parameters found: ", clf.best_params_)
print("Best cross-validation score: {:.2f}".format(clf.best_score_))

Best parameters found:  {'max_depth': 30, 'min_samples_split': 2}
Best cross-validation score: 0.99


**Assessing Model Performance in the Face of Imbalanced Data**


Given the dataset's likely imbalanced nature, special attention is needed to evaluate the model's ability to predict less frequent events correctly. Here, we use the Naive Bayes model as an example:

In [17]:
# Naive Bayes model for quick evaluation
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

# Performance metrics
report = classification_report(y_test, y_pred, target_names=['Non-CLABSI', 'CLABSI'])
print(report)


              precision    recall  f1-score   support

  Non-CLABSI       0.99      0.96      0.98      2827
      CLABSI       0.04      0.19      0.06        21

    accuracy                           0.96      2848
   macro avg       0.52      0.58      0.52      2848
weighted avg       0.99      0.96      0.97      2848



**Context-Specific Evaluation Metrics**


Medical predictive modeling must carefully weigh the cost of false positives against false negatives. In this context, metrics that provide insight into these aspects are critical. We utilize the K-Nearest Neighbors model to illustrate this:

In [18]:


# K-Nearest Neighbors model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
y_proba = knn.predict_proba(X_test)[:, 1]

# AUC for model evaluation
auc_score = roc_auc_score(y_test, y_proba)
print("AUC: {:.2f}".format(auc_score))


AUC: 0.87


**Handling Imbalanced Data with SMOTE**

To address the challenge of imbalanced data, which is common in medical datasets where one class (e.g., patients with CLABSI) is significantly underrepresented, we will use SMOTE. This technique generates synthetic samples from the minority class to achieve balance between the classes, which can improve model performance, especially for algorithms that are sensitive to class imbalance.



**Implementing SMOTE with Decision Tree Classifier**

We choose the Decision Tree model to demonstrate the effect of SMOTE, as decision trees can often overfit to the majority class in their default settings.



In [19]:
# Preparing the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Applying SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)



# Training the Decision Tree model on SMOTE-enhanced data
dt = DecisionTreeClassifier(max_depth=10, random_state=42)
dt.fit(X_train_smote, y_train_smote)
y_pred = dt.predict(X_test)

# Evaluating the model
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, dt.predict_proba(X_test)[:, 1]))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.98      2827
           1       0.15      0.76      0.25        21

    accuracy                           0.97      2848
   macro avg       0.58      0.87      0.62      2848
weighted avg       0.99      0.97      0.98      2848

ROC AUC Score: 0.9117775868748632



1. **High Recall (0.76)**: Your model successfully identifies 76% of actual CLABSI cases, which is crucial in medical applications where missing a true positive can have serious consequences.
2. **Low Precision (0.15)**: The model produces many false positives; only 15% of predictions for CLABSI are correct. This means a lot of the predictions indicating CLABSI are false alarms.
3. **ROC AUC Score (0.9118)**: This high score suggests that the model is generally good at distinguishing between patients with and without CLABSI, but it might be setting the classification threshold too low, leading to many false positives.

**Interpretation:**
- **Trade-off Between Precision and Recall**: The model is tuned to avoid missing CLABSI cases (high recall) at the cost of higher false positives (low precision). This might be acceptable in some medical scenarios where failing to detect a condition is worse than false alarms.
  
- **Model and Threshold Adjustment**: Adjusting the classification threshold to reduce false positives or experimenting with different models or tuning parameters to find a better balance between precision and recall.

The model's current setup prioritizes minimizing missed cases of CLABSI, which is typical in medical diagnostics. However, depending on the consequences of false positives in your specific application,the model is finetuned or its decision threshold to balance the outcomes better.